In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.models import Model
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad,Adam
from keras.callbacks import TensorBoard

import keras
# 数据准备
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,# ((x/255)-0.5)*2  归一化到±1之间
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

这里用到的数据集和之前都不同，之前用的是一些公共的、Keras内置的数据集，这次用到的是自己准备的数据集。由于数据的图像大小比较大，不适合一次全部载入到内存中，所以使用了flow_from_directory方法来按批次从硬盘读取图像数据，并实时进行图像增强

In [ ]:
train_generator = train_datagen.flow_from_directory(directory='./chnData/train',
                                  target_size=(32,32),#Inception V3规定大小
                                  batch_size=64)
val_generator = val_datagen.flow_from_directory(directory='./chnData/test',
                                target_size=(32,32),
                                batch_size=64)

首先我们需要加载骨架模型，这里用的InceptionV3模型，其两个参数比较重要，一个是weights，如果是'imagenet',Keras就会自动下载已经在ImageNet上训练好的参数，如果是None，系统会通过随机的方式初始化参数，目前该参数只有这两个选择。另一个参数是include_top，如果是True，输出是1000个节点的全连接层。如果是False，会去掉顶层，输出一个8 \* 8 \* 2048的张量。 

ps：在keras.applications里还有很多其他的预置模型，比如VGG，ResNet，以及适用于移动端的MobileNet等。大家都可以拿来玩玩。

一般我们做迁移训练，都是要去掉顶层，后面接上各种自定义的其它新层。这已经成为了训练新任务惯用的套路。
输出层先用GlobalAveragePooling2D函数将8 \* 8 \* 2048的输出转换成1 \* 2048的张量。后面接了一个1024个节点的全连接层，最后是一个17个节点的输出层，用softmax激活函数。

In [ ]:
#ResNet Block
def resnet_block(inputs,num_filters=16,
                  kernel_size=3,strides=1,
                  activation='relu'):
    x = Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',
           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if(activation):
        x = Activation('relu')(x)
    return x

In [ ]:
# 建一个20层的ResNet网络 
def resnet_v1(input_shape):
    inputs = Input(shape=input_shape)# Input层，用来当做占位使用
    
    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out：32*32*16
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out:16*16*32
    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
        x = Activation('relu')(x)
    # out:8*8*64
    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    # out:4*4*64
    y = Flatten()(x)
    # out:1024
    outputs = Dense(3562,activation='softmax',
                    kernel_initializer='he_normal')(y)
    
    #初始化模型
    #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
model = resnet_v1((32,32,3))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [ ]:
tb = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=False,  # 是否存储网络结构图
                 write_grads=False, # 是否可视化梯度直方图
                 write_images=False,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)
checkpoint = ModelCheckpoint(filepath='./chnData_resnet_ckpt.h5',monitor='val_acc',
                             verbose=1,save_best_only=True)
callbacks = [tb,checkpoint]

In [ ]:
history_tl = model.fit_generator(generator=train_generator,
                    steps_per_epoch=800,#800
                    epochs=20,#2
                    validation_data=val_generator,
                    validation_steps=12,#12
                    class_weight='auto',
                    callbacks=callbacks
                    )
model.save('./chnData_resnet.h5')
